In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import json

In [7]:
import string
ascii = set(string.printable)   

def remove_non_ascii(s):
    st = ''
    for x in s:
        if x in ascii:
            st += x
    return st

In [8]:
def clean(s):
    s = s.strip()
    s = s.strip(",")
    s = s.strip("/")
    s = s.strip("\\")
    s = s.strip("---")
    s = s.strip()
    s = remove_non_ascii(s)
    return s

In [2]:
data = []
for line in open('33318.json', 'r'):
    data.append(json.loads(line))
    
df = pd.io.json.json_normalize(data)

<ipython-input-2-b58cbde9ee4a>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(data)


In [3]:
docs = {}

for i in range(len(df)):
    dp = {}
    #dp['index'] = i
    dp['seed'] = df['seed_name'][i]  ## query product title
    
    del_titles = []
    active_titles = []
    
    s = len(df['detailed'][i])
    for l in range(s):
        if df['detailed'][i][l]['status'] == 'deleted':
            del_titles.append(df['detailed'][i][l]['title'])   ## title of deleted prod
            
        if df['detailed'][i][l]['status'] == 'active':
            active_titles.append(df['detailed'][i][l]['title'])  ## title of accepted prod
    
    dp['active_titles'] = active_titles
    dp['deleted_titles'] = del_titles
    
    docs[i] = dp

In [68]:
df_processed = pd.DataFrame(docs.values())
print(len(df_processed))
df_processed.head()

17000


,seed,active_titles,deleted_titles
0,Clarks Desert Trek Black Suede 1,[Clarks Desert Trek Black Suede 1],"[, 883956040685, (DARK WOOD/DARK JAVA) [8.5]---]"
1,Haflinger GZ Classic Grizzly Bordo,"[Haflinger GZ Classic Grizzly Bordo, Haflinger...",[Mens Birkenstock Zermatt Clog - Mocha Mocha |...
2,Nike Air Zoom Terra Kiger 7 Limelight/Off-Noir...,[Nike Air Zoom Terra Kiger 7 Limelight/Off-Noi...,[Nike Air Monarch IV Training Shoe - Men's Whi...
3,adidas Originals Trefoil Tee White/Hazy Rose,[adidas Originals Trefoil Tee White/Hazy Rose],"[Men's adidas Orlando City Away Soccer Jersey""..."
4,Save the Duck Kids Flee (Little Kids/Big Kids)...,[Save the Duck Kids Flee (Little Kids/Big Kids...,[Save The Duck Randolph Bomber Jacket NAVY BLU...


In [5]:
del data
del df

In [69]:
df_processed['seed'] = [ '' if x is np.NaN else x for x in df_processed['seed'] ]
df_processed['active_titles'] = [ '' if x is np.NaN else x for x in df_processed['active_titles'] ]
df_processed['deleted_titles'] = [ '' if x is np.NaN else x for x in df_processed['deleted_titles'] ]

In [70]:
## removing special chars => !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`
import string
import re

PUNCT_TO_REMOVE = string.punctuation
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace("-","")
# PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace(".","")
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace("&","")

def remove_special_chars(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [71]:
df_processed['seed'] = df_processed['seed'].astype(str)
df_processed['active_titles'] = df_processed['active_titles'].astype(str)
df_processed['deleted_titles'] = df_processed['deleted_titles'].astype(str)

In [72]:
df_processed['seed'] = df_processed['seed'].apply(lambda text: remove_special_chars(text))
df_processed['active_titles'] = df_processed['active_titles'].apply(lambda text: remove_special_chars(text))
df_processed['deleted_titles'] = df_processed['deleted_titles'].apply(lambda text: remove_special_chars(text))

In [73]:
df_processed['seed'] = df_processed['seed'].str.replace('\d+', '')
df_processed['active_titles'] = df_processed['active_titles'].str.replace('\d+', '')
df_processed['deleted_titles'] = df_processed['deleted_titles'].str.replace('\d+', '')

In [74]:
df_processed.head()

,seed,active_titles,deleted_titles
0,Clarks Desert Trek Black Suede,Clarks Desert Trek Black Suede,DARK WOODDARK JAVA ---
1,Haflinger GZ Classic Grizzly Bordo,Haflinger GZ Classic Grizzly Bordo Haflinger G...,Mens Birkenstock Zermatt Clog - Mocha Mocha U...
2,Nike Air Zoom Terra Kiger LimelightOff-NoirLa...,Nike Air Zoom Terra Kiger LimelightOff-NoirLa...,Nike Air Monarch IV Training Shoe - Mens White...
3,adidas Originals Trefoil Tee WhiteHazy Rose,adidas Originals Trefoil Tee WhiteHazy Rose,Mens adidas Orlando City Away Soccer Jersey - ...
4,Save the Duck Kids Flee Little KidsBig Kids Wa...,Save the Duck Kids Flee Little KidsBig Kids Wa...,Save The Duck Randolph Bomber Jacket NAVY BLUE...


In [75]:
sentences = []
for row in df_processed['seed']:
    row = clean(str(row))
    sentences.append(str(row))
    
for row in df_processed['active_titles']:
    row = clean(str(row))
    sentences.append(str(row))

for row in df_processed['deleted_titles']:
    row = clean(str(row))
    sentences.append(str(row))
    
print(len(sentences))

51000


In [76]:
tfidf = TfidfVectorizer()
tfidf_rep = tfidf.fit_transform(sentences)

In [77]:
tfidf_rep

<51000x19272 sparse matrix of type '<class 'numpy.float64'>'
	with 408438 stored elements in Compressed Sparse Row format>

In [108]:
e1 = tfidf.transform(['Clarks Desert Trek Black Suede'])

In [102]:
e2 = tfidf.transform(['Haflinger GZ Classic Grizzly Bordo Haflinger GZ Clog Slipper Women BORDO  US  EU'])

In [103]:
e1 = e1.toarray()
e2 = e2.toarray()

In [104]:
e1 = np.squeeze(np.asarray(e1))

e2 = np.squeeze(np.asarray(e2))

In [92]:
df_processed['seed'].iloc[0]

'Clarks Desert Trek Black Suede '

In [94]:
df_processed['active_titles'].iloc[1]

'Haflinger GZ Classic Grizzly Bordo Haflinger GZ Clog Slipper Women BORDO  US  EU'

In [105]:
cos_sim = dot(e1, e2)/(norm(e1)*norm(e2))

In [106]:
cos_sim

0.0

### Calculating Precision / Recall / F1 metrics  

##### Threshold = 0.75  

In [109]:
# threshold for classification
threshold = 0.75

In [111]:
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0

for i in range(len(df_processed)):
    
    s_q = df_processed['seed'][i]
    s_q = clean(s_q)
    w1 = tfidf.transform([s_q])
    
    for l in range(len(df_processed['active_titles'][i])):
        s_a = df_processed['active_titles'][i][l]
        s_a = clean(s_a)
        w2 = tfidf.transform([s_a])
        cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))
        
        if cos_sim >= threshold:         ## classify as accepted match  (good) (True positive)
            true_positive += 1
            
        elif cos_sim < threshold:        ## classify as rejected match  (bad)  (False negative)
            false_negative += 1
        
    for l in range(len(df_processed['deleted_titles'][i])):
        s_d = df_processed['deleted_titles'][i][l]
        s_d = clean(s_d)
        w2 = tfidf.transform([s_d])
        cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))
        
        if cos_sim >= threshold:          ## classify as accepted match   (bad)  (False positive)
            false_positive += 1
        
        elif cos_sim < threshold:          ## classify as rejected match   (good)  (True Negative)
            true_negative += 1
        
        
    if (i % 1000 == 0):
        print(i/1000)

ValueError: dimension mismatch